Normalização do modelo BCS não linear

In [303]:
import matplotlib.pyplot as plt
#import importlib
import numpy as np
#import subrotinas

#importlib.import_module('subrotinas')
%matplotlib tk


exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open('subrotinas.py', "rb").read(), 'subrotinas.py', 'exec')) #% Roda arquivo

### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>

### Saídas
pin $\in$ ($40, 60.5) \cdot 10^6$ <br>
H $\in$ ($350, 1000$) <br>

Função para retornar os valores de xc e x0 dado que<br>
$x_{min}<x<x_{max}$ <br>

Nova variável<br>

$xb=\frac{x-x0}{xc}$

Assim <br>

$x=xb\cdot xc+x0$

Fazendo com que a nova variável $0<xb<1$ <br>


In [304]:
# Rotinas

Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [305]:
# Valores máximos e mínimos para normalização
#Entradas
f_lim=(30,75); zclim=(0,100);pmlim=(1e6,2e6);
#pbhlim=(100000,8.5e6);
pbhlim=(1e6,8.5e6);pwhlim=(2e6,5.2e6); qlim=(12/3600,55/3600)
H_lim=(350,1000);

#Hlim e Pin modificado
pinlim=(-3.28e12,-5.2e8)
H_lim=(-9.8e9,-110196);
#pinlim=(40e6,60.5e6)

In [306]:
pm=2e6; #Simplificando pm fixo
unorm=np.array([Fnorm(f_lim),Fnorm(zclim)])
#xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim), unorm[0,:],unorm[1,:]])
xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim)])
ynorm=np.array([Fnorm(pinlim),Fnorm(H_lim)])
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

Definindo as variáveis simbólicas

In [307]:
# Criando simbolica
from casadi import *
nx = 3; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena

In [308]:
[pbh,pwh,q]=[x[0],x[1],x[2]]
x
#xnorm=np.array([[0,1],[0,1],[0,1]])

MX(x)

In [309]:
pbh = (x[0]-xnorm[0,0])/xnorm[0,1]
pwh = (x[1]-xnorm[1,0])/xnorm[1,1]
q = (x[2]-xnorm[2,0])/xnorm[2,1]
# pwh = AplicaEscala(x[1],1);
# q = AplicaEscala(x[2],2)
fq = u[0]
zc = u[1]
#zc = AplicaEscala(x[4],4)

In [310]:
# Entradas

fqref = u[0]*unorm[0,1]+unorm[0,0]; zcref = u[1]*unorm[1,1]+unorm[1,0]
pm=2e6
#fq=fqref; zc=zcref;
#pm=(pm-xnorm[0,0])/xnorm[0,1]
#pm = u[2];

In [311]:
# Calculo do HEAD e delta de press�o
q0 = q/Cq*(f0/fq);
H0 = -1.2454e6*q0**2 + 7.4959e3*q0 + 9.5970e2;
H = CH*H0*(fq/f0)**2; # Head
H

MX(@1=u[0], @2=((((x[2]-0.00333333)/0.0119444)/0.937982)*(60/@1)), ((0.99925*(959.7+((-1245400*sq(@2))+(7495.9*@2))))*sq((@1/60))))

Normalizando H (depende de u[0] e x[2])
Variação de H com u[0] e x[2] normalizados

In [312]:
# Definimos duas variáveis para teste nas funções H e Pin
ui=np.arange(0.1,1.01,0.1)
xi=np.arange(0.1,1.01,0.1)

z=np.zeros((len(ui),len(xi)))
funcH=Function('funcH',[u,x],[H])
for i in range (0,len(ui)):
    for j in range(0,len(xi)):
        z[i,j]=funcH([ui[i],0],[0,0,xi[j]])
U,X=np.meshgrid(ui,xi)
#z=funcH([U,0],[0,0,X])
# Plot the surface.
from matplotlib import cm
from matplotlib.ticker import LinearLocator
figH, ax = plt.subplots(subplot_kw={"projection": "3d"})
norm = cm.colors.Normalize(vmin=0.1, vmax=1)
surf = ax.plot_surface(U, X, z , cmap=cm.coolwarm, norm=norm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
#ax.set_zlim(0.19, 1)
ax.set_xlabel('frequencia')
ax.set_ylabel('vazão')
ax.set_zlabel('H')
ax.zaxis.set_major_locator(LinearLocator(5))
# A StrMethodFormatter is used automatically
ax.zaxis.set_major_formatter('{x:.02f}')
# Add a color bar which maps values to colors.
figH.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [313]:
min_max_H=[np.amin(z), np.amax(z)]
Hlim =min_max_H
ynorm=np.array([Fnorm(pinlim),Fnorm(H_lim)])
H=(H-ynorm[1,0])/ynorm[1,1]
z=np.zeros((len(ui),len(xi)))
funcH=Function('funcH',[u,x],[H])
for i in range (0,len(ui)):
    for j in range(0,len(xi)):
        z[i,j]=funcH([ui[i],0],[0,0,xi[j]])
U,X=np.meshgrid(ui,xi)
figH, ax = plt.subplots(subplot_kw={"projection": "3d"})
norm = cm.colors.Normalize(vmin=0.1, vmax=1)
surf = ax.plot_surface(U, X, z , cmap=cm.coolwarm, norm=norm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
#ax.set_zlim(0.19, 1)
ax.set_xlabel('frequencia')
ax.set_ylabel('vazão')
ax.set_zlabel('H')
ax.zaxis.set_major_locator(LinearLocator(5))
# A StrMethodFormatter is used automatically
ax.zaxis.set_major_formatter('{x:.02f}')
# Add a color bar which maps values to colors.
figH.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [314]:
c()

NameError: name 'c' is not defined

In [ ]:
H=(H-ynorm[1,0])/ynorm[1,1]

In [ ]:
Pp = rho*g*H;
# Calculo da press�o de intake
F1 = 0.158*((rho*L1*q**2)/(D1*A1**2))*(mu/(rho*D1*q))**(1/4);
F2 = 0.158*((rho*L2*q**2)/(D2*A2**2))*(mu/(rho*D2*q))**(1/4);

funcPin=Function('funcPin',[x],[x[0]-rho*g*h1-F1])
funcPin

In [ ]:
zp=np.zeros((len(ui),len(xi)))
for i in range (0,len(ui)):
    for j in range(0,len(xi)):
        zp[i,j]=funcPin([ui[i],0,xi[j]])



In [ ]:
# #Limites de Pin
# figPin, axP = plt.subplots(subplot_kw={"projection": "3d"})
# norm = cm.colors.Normalize(vmin=0.1, vmax=1)
# surfPin = ax.plot_surface(U, X, zp , cmap=cm.coolwarm,# norm=norm,
#                        linewidth=0, antialiased=False)
# # Customize the z axis.
# #axP.set_zlim(0.19, 1)
# axP.set_xlabel('Pbh')
# axP.set_ylabel('q')
# axP.set_zlabel('Pin')
# #axP.zaxis.set_major_locator(LinearLocator(5))
# # A StrMethodFormatter is used automatically
# #axP.zaxis.set_major_formatter('{x:.02f}')
#
# # Add a color bar which maps values to colors.
# figPin.colorbar(surfPin, shrink=0.5, aspect=5)
# plt.show()

In [ ]:
zp


In [ ]:
c()


In [ ]:
P0 = -2.3599e9*q0**3 -1.8082e7*q0**2 +4.3346e6*q0 + 9.4355e4;
pin = pbh - rho*g*h1 - F1
F1

Normalizando pin

In [ ]:
pin=(pin-ynorm[0,0])/ynorm[0,1]

In [ ]:
P = Cp*P0*(fq/f0)**3; # Potencia
I = Inp*P/Pnp;      # Corrente

In [ ]:
# Vazao do reservatorio e vazao da choke
qr  = PI*(pr - pbh);
qc  = Cc*(zc/100)*sign((pwh - pm))*sqrt(casadi.fabs(pwh - pm));
dpbhdt = b1/V1*(qr - q);
dpwhdt = b2/V2*(q - qc);
dqdt = 1/M*(pbh - pwh - rho*g*hw - F1 - F2 + Pp);

Aplicar escala nas derivadas

In [ ]:
dpbhdt=dpbhdt/xnorm[0,1]
dpwhdt=dpwhdt/xnorm[1,1]
dqdt=dqdt/xnorm[2,1]


In [ ]:
# Função casadi
dxdt = vertcat(dpbhdt,dpwhdt,dqdt)
Eq_Estado = Function('Eq_Estado',[x,u],[dxdt],
                     ['x','u'],['dxdt'])

y=vertcat(pin,H);
ny = y.size1()

In [ ]:
# Equações algébricas
sea_nl = Function('sea_nl',[x,u],[y,pin,H],\
                  ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

In [ ]:
BCS={
     'x': x,'u': u, 'y': y,
     # 'x': vertcat(pbh,pwh,q),'u': vertcat(fq,zc), 'y': y,
     'nx': nx,'nu': nu,'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u),
                   'sea_nl': sea_nl
                   }
}

In [ ]:
#% Função objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u'])
J = sum1(dxdt_0**2)

In [ ]:
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;

In [ ]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
solver = nlpsol('solver', 'ipopt', MMQ, opt)

In [ ]:
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }
# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])

In [ ]:
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],\
     [sol['x'],yss[0]],\
     ['x0','uss'],['xss','yss']);

BCS['Estacionario'] = Estacionario;



In [ ]:
# Definindo regime permanente
f_ss,zc_ss= (np.array([50, 50]))
uss = np.array([[f_ss],[zc_ss]]); # Entradas do estacionario
#% Calculo do estacionario
#x0 = [0.2,0.5,0.5,0.5,0.5]
#x0 = np.array([8311024.82175957,2990109.06207437,0.00995042241351780,50,50]);

x0 = np.array([8311024.82175957,2990109.06207437,0.00995042241351780]);
x0

In [ ]:
x0_n=normalizar_x(x0,xnorm)
x0_n

In [ ]:
uss_n=normalizar_u(uss,unorm)
uss_n

In [ ]:
[xss,yss]=Estacionario(x0_n,uss_n)
xss

In [ ]:
yss

In [ ]:
# args['lbx'][3] = uss[0]
# args['ubx'][3] = uss[0]  # bounds freq. solver
# args['lbx'][4] = uss[1]
# args['ubx'][4] = uss[1]   # bounds zc solver
#sol=solver('x0',x0, 'lbx', args['lbx'], 'ubx', args['ubx'], 'p', uss);
#sol=solver(x0=x0_n,lbx=args['lbx'], ubx=args['ubx'], p=uss_n)
sol_n=solver(x0=x0_n, p=uss_n)
#sol_n=solver(x0=x0, p=uss)
#xss_n=sol_n['x']
xss=sol_n['x']

In [ ]:
#xss = desnormalizar(sol_n['x'],xnorm)
xss

xss correto = <br>
6.97226e+06<br>
2.86974e+06<br>
0.0130564

DM([8.31102e+06, 2.99011e+06, 0.00995042])


yss

In [ ]:
out_ss=sea_nl(xss,uss)
[xss,yss]=Estacionario(x0,uss)
xss

In [ ]:
[xss_n,yss_n]=Estacionario(x0_n,uss_n)
xss_n

In [ ]:
yss


In [ ]:
desnormalizar_x(xss,xnorm)

In [ ]:
out_ss=sea_nl(desnormalizar_x(xss,xnorm),uss)[0]
np.array(out_ss).T

In [ ]:
x0.reshape(3,1)
c()

In [ ]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = 2;      #% [pm]
tg = 2;      #% MV target
#% Parametros
ts = 1;

In [ ]:
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'][0:3], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x[0:3],u) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto para integração da Eq_estado
opt = {'tf':ts,'t0':0};   #% opções do integrador

In [ ]:
int_odes = integrator('int_odes','cvodes',sedo,opt);
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv);

In [ ]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);

In [ ]:
# Inicializa��o das variaveis

xmk = xss;           # Estados
xpk = xss;

In [ ]:
uk_1=uss[mv];
#xpk=normalizar(xss,xnorm)
xpk=(xss-xnorm[:,0])/xnorm[:,1]
#uk_1 = normaliza_u(uss[mv],unorm)
uk_1=(uss[mv]-unorm[:,0])/unorm[:,1]
# MVS
# Aloca��o de variaveis
Xk = np.zeros((nx,1))
Xk

In [ ]:
Yk = np.zeros((npv,1))
Uk = np.zeros((nmv,1))
Ymk = Yk
Ys = Yk
Ymin = Yk
Ymax = Yk

### Simulando cenários

In [ ]:
tsim = 50;
nsim=int(round(tsim/ts)+1)
[xpk,ypk] = Modelo_Predicao(xss_n,uss_n)

In [ ]:
Xk

In [ ]:
xi=(np.arange(0,int(nsim*ts)-1,ts));
uk_1 = np.array([[60], [70]]);
#ypk=desnormalizar_y(np.array(Eq_medicao(xpk,uk_1)),ynorm)
ypk=np.array(Eq_medicao(xpk,uk_1))*ynorm[:,1]+ynorm[:,0]
#uk_1n = normaliza_u(uk_1,unorm)
uk_1n=(uk_1.T-unorm[:,0])/unorm[:,1]

In [ ]:
Uk= uss

xpk*xnorm[:,1]+xnorm[:,0]
xpk

In [ ]:
Xk=xpk*xnorm[:,1]+xnorm[:,0]
Xk

In [ ]:
Yk=ypk


In [ ]:
y2=sea_nl(desnormalizar(xpk,xnorm),uk_1)[0]
desnormalizar(xpk,xnorm)
ypk

In [ ]:
y2

In [ ]:
for k in range(1,nsim-1):
    #print('Tempo:',k*ts, 'k=',k)
    [xpk,ypk] = Modelo_Predicao(xpk,uk_1n)
    print(desnormalizar(xpk,xnorm))
    Xk = hcat([Xk,desnormalizar(xpk,xnorm)])
    print(Xk.shape)
    ypk=Eq_medicao(xpk,uk_1)
    #y2=sea_nl(desnormalizar(xpk,xnorm),uk_1)
    #y2
    Yk = hcat([Yk,desnormalizar_y(np.array(ypk),ynorm)]);
    #Yk = hcat([Yk,y2]);
    Uk = hcat([Uk,uk_1])

    #Ymk = hcat([Ymk,ymk]);
    #print("Xk,Yk,Uk,xi shape ="+str(Xk.shape)+" "+str(Yk.shape)+" "+str(Uk.shape)+" "+str(xi.shape))
print("Xk shape ="+str(Xk.shape))
print("Uk shape ="+str(Uk.shape))
print("Yk shape ="+str(Uk.shape))
print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")

#Xk[2,:]

## Produção dos gráficos

In [ ]:
fig1=plt.figure()
label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
for iy in range(0,npv):
    ax = fig1.add_subplot(npv,1,iy+1)
    #print(iy)
    if iy == 0: # Pin
        ax.plot(xi,(Yk[iy,:].T), label='Medição')
        #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(40,62))
        plt.grid(True)
    else: # H
        ax.plot(xi,Yk[iy,:].T, label='Medição')
        #ax.plot(xi,Ymk[iy,:].T,label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(580, 850))
        plt.grid(True)
#ax.plot(xi,Yk[2,:].T, label='EKF')
ax.legend();
ax.set_xlabel('Time (nT)')
fig1.show()

In [ ]:
Yk[1,:].T.shape

In [ ]:
#% Restrição
umin  = np.array([35, 0]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)'];
for iu in range(0,nmv):
    ax2=fig2.add_subplot(nmv,1,iu+1)
    ax2.plot(xi,Uk[iu,:].T, label='Medição')
    ax2.plot([1,nsim],[umin[iu], umin[iu]],'--r')
    ax2.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax2.set_ylabel(label[iu])
    ax2.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        ax2.set(ylim=(30, 70))
        print(iu)
    plt.grid(True)

In [ ]:
fig3=plt.figure()
label = ['Pbh','Pwh','q'];
for iu in range(0,nx):
    ax3=fig3.add_subplot(nx,1,iu+1)
    ax3.plot(xi,Xk[iu,:].T, label='Medição')
    #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
    #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax3.set_ylabel(label[iu])
    ax3.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        #ax2.set(ylim=(30, 70))
        print(iu)
    plt.grid(True)

In [ ]:
# fig1=plt.figure()
# label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
# Yk=np.array(Yk).reshape(100,2)
# plt.plot(xi,np.array(Yk[:,0]))
# plt.plot(xi,np.array(Yk[:,1]))

In [ ]:
# ax = fig1.add_subplot(2,1,1)
# #ax.set_xlabel('Time (nT)')
# ax.plot(xi,np.array(Yk[:,0]))
# ax = fig1.add_subplot(2,1,2)
# ax.plot(xi,Yk[:,1])
# plt.grid(True)
# fig1.show()

In [ ]:
[Yk[:,0].shape, xi.shape]